Задание к лекции "Основы веб-скрапинга и работы с API"

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [53]:
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/all/'

In [57]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import regex

In [59]:
def get_all_info(url, keywords, pages):
    habr_posts = pd.DataFrame()
    
    for i in range(pages):
        params = 'page' + str(i + 1)
        res = requests.get(url+params)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text, 'html.parser')
        title_blocks = soup.find_all('h2', class_='post__title')
        text_blocks = soup.find_all('div', class_='post__text post__text-html post__text_v1')
        time_block = soup.find_all('span', class_='post__time')
        titles = list(map(lambda x: x.find('a', class_='post__title_link').text, title_blocks))
        linkes = list(map(lambda x: x.find('a').get('href'), title_blocks))
        texts= list(map(lambda x: x.get_text(),text_blocks))
        times = list(map(lambda x: x.get_text(), time_block))
        infos = [list(tup) for tup in zip(times, titles, texts, linkes)]
        for j in range(len(infos)):
            if regex.search(r"\L<words>", infos[j][1], words=keywords) or\
            regex.search(r"\L<words>", infos[j][2], words=keywords):
                row = {'Время': infos[j][0], 'Заголовок': infos[j][1], 'Текст': infos[j][2], 'Ссылка': infos[j][3]}
                habr_posts = pd.concat([habr_posts, pd.DataFrame([row])]) 
                 
    return habr_posts

In [60]:
all_links = get_all_info(URL, KEYWORDS, 30)
all_links

,Время,Заголовок,Текст,Ссылка
0,26 октября 2020 в 21:35,"Рудиментарные дела. Кого, чему, зачем и что не...","\n Привет, друзья! Хорошая новость для мо...",https://habr.com/ru/post/525168/
0,22 октября 2020 в 11:47,Мелкая питонячая радость #11: реактивное прогр...,\nВ данном руководстве мы рассмотрим подключен...,https://habr.com/ru/post/523878/
